# Ideal-gas properties 

See the appendix of https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=933725 for the mathematical representations. They can be summarized like so:

$
u_{\rm ig} = RT\left((1/T)\left(\frac{\partial\alpha_{\rm ig}}{\partial(1/T)}\right)_{\delta}\right) = RT\left(\tau\left(\frac{\partial\alpha_{\rm ig}}{\partial\tau}\right)_{\delta}\right)
$

$
h_{\rm ig} = RT\left(1+(1/T)\left(\frac{\partial\alpha_{\rm ig}}{\partial(1/T)}\right)_{\delta}\right) = RT\left(1+\tau\left(\frac{\partial\alpha_{\rm ig}}{\partial\tau}\right)_{\delta}\right)
$

$
s_{\rm ig} = R\left((1/T)\left(\frac{\partial\alpha_{\rm ig}}{\partial(1/T)}\right)_{\delta}-\alpha_{\rm ig}\right) = R\left(\tau\left(\frac{\partial\alpha_{\rm ig}}{\partial\tau}\right)_{\delta}-\alpha_{\rm ig}\right)
$

The precise reference state used in Table A-22 of Moran and Shapiro is unknown, but they are based on the gas tables from the 1950s at the then National Bureau of Standards (now NIST). According to [a summary report from one year later](https://www.govinfo.gov/content/pkg/GOVPUB-C13-89baf9f9b4a43e5f25820bd51b0f3f11/pdf/GOVPUB-C13-89baf9f9b4a43e5f25820bd51b0f3f11.pdf), the enthalpy and Gibbs energy (and therefore also the entropy because $g=h-Ts$) are set to 0 at 0 K. This differs to the reference state used in [the pseudo-pure Air EOS](https://coolprop.org/fluid_properties/fluids/Air.html#fluid-air), but the offset is identical, as shown here:

In [ ]:
import CoolProp.CoolProp as CP

In [ ]:
# Some check values from Moran & Shapiro, 6th edition, Table A-22
Ts = [200, 440, 740]
hs = [199.97, 441.61, 756.44]
ss = [1.29559, 2.08870, 2.63280]
us = [142.56, 315.30, 544.02]

for T, h0, s0, u0 in zip(Ts, hs, ss, us):
    AS = CP.AbstractState("HEOS", "Air")
    # Use the density obtained from Z=1
    rho = 101325/(CP.PropsSI('molemass','Air')*T)
    AS.update(CP.DmolarT_INPUTS, rho, T)
    R = AS.gas_constant()/AS.molar_mass()
    RT = R*T
    ucalc_kJkg = RT*(AS.tau()*AS.dalpha0_dTau())/1000 # kJ/kg
    hcalc_kJkg = RT*(1+AS.tau()*AS.dalpha0_dTau())/1000 # kJ/kg
    scalc_kJkgK = R*(AS.tau()*AS.dalpha0_dTau()-AS.alpha0())/1000 # kJ/kg/K
    print('T:', T, 'u:', ucalc_kJkg, 'h:', hcalc_kJkg, 's:', scalc_kJkgK, 'deltau:', ucalc_kJkg-u0, 'deltah:', hcalc_kJkg-h0, 'deltas:', scalc_kJkgK-s0)

As of version 7.2, you can also get the ideal-gas properties directly from the model without additional work:

In [ ]:
for T, h0, s0, u0 in zip(Ts, hs, ss, us):
    AS = CP.AbstractState("HEOS", "Air")

    # Use the density obtained from Z=1
    rho = 101325/(CP.PropsSI('molemass','Air')*T)
    AS.update(CP.DmolarT_INPUTS, rho, T)
    R = AS.gas_constant()/AS.molar_mass()
    RT = R*T
    ucalc_kJkg = AS.umass_idealgas()/1000 # kJ/kg
    hcalc_kJkg = AS.hmass_idealgas()/1000 # kJ/kg
    scalc_kJkgK = AS.smass_idealgas()/1000 # kJ/kg/K
    print('T:', T, 'u:', ucalc_kJkg, 'h:', hcalc_kJkg, 's:', scalc_kJkgK, 'deltau:', ucalc_kJkg-u0, 'deltah:', hcalc_kJkg-h0, 'deltas:', scalc_kJkgK-s0)

Or with the high-level interface (less efficient computationally)

In [ ]:
for T, h0, s0, u0 in zip(Ts, hs, ss, us):
    # Use the density obtained from Z=1
    rho = 101325/(CP.PropsSI('molemass','Air')*T)
    ucalc_kJkg = CP.PropsSI('Umass_idealgas','T|phase_gas',T,'Dmolar',rho,'Air')/1000 # kJ/kg
    hcalc_kJkg = CP.PropsSI('Hmass_idealgas','T|phase_gas',T,'Dmolar',rho,'Air')/1000 # kJ/kg
    scalc_kJkgK = CP.PropsSI('Smass_idealgas','T|phase_gas',T,'Dmolar',rho,'Air')/1000 # kJ/kg/K
    print('T:', T, 'u:', ucalc_kJkg, 'h:', hcalc_kJkg, 's:', scalc_kJkgK, 'deltau:', ucalc_kJkg-u0, 'deltah:', hcalc_kJkg-h0, 'deltas:', scalc_kJkgK-s0)